In [1]:
race_results = {
    'id': 'rvec2024',
    'track': 'Santorini',
    'laps': 86,
    'players': {
        'Stingox'  : {'car': 'Toyeca', 'laps': ['00:31.585', '00:31.585', '00:31.585', '00:31.585', '00:31.585', '00:31.585', '00:31.585']},
        'Famous'   : {'car': 'Toyeca', 'laps': ['00:31.679', '00:31.679', '00:31.679', '00:31.679', '00:31.679', '00:31.679', '00:31.679']},
        'Kiki'     : {'car': 'Toyeca', 'laps': ['00:31.710', '00:31.710', '00:31.710', '00:31.710', '00:31.710', '00:31.710', '00:31.710']},
        'Kilabarus': {'car': 'Toyeca', 'laps': ['00:31.719', '00:31.719', '00:31.719', '00:31.719', '00:31.719', '00:31.719', '00:31.719']},
        'Powerate' : {'car': 'Toyeca', 'laps': ['00:31.791', '00:31.791', '00:31.791', '00:31.791', '00:31.791', '00:31.791', '00:31.791']},
        'Gforce'   : {'car': 'Toyeca', 'laps': ['00:31.833', '00:31.833', '00:31.833', '00:31.833', '00:31.833', '00:31.833', '00:31.833']},
        'Macacosky': {'car': 'Toyeca', 'laps': ['00:31.863', '00:31.863', '00:31.863', '00:31.863', '00:31.863', '00:31.863', '00:31.863']},
        'Ahma'     : {'car': 'Toyeca', 'laps': ['00:31.890', '00:31.890', '00:31.890', '00:31.890', '00:31.890', '00:31.890', '00:31.890']},
        'Nickurn'  : {'car': 'Toyeca', 'laps': ['00:31.933', '00:31.933', '00:31.933', '00:31.933', '00:31.933', '00:31.933', '00:31.933']},
        'Lampaert' : {'car': 'Toyeca', 'laps': ['00:32.024', '00:32.024', '00:32.024', '00:32.024', '00:32.024', '00:32.024', '00:32.024']},
        'Iurac'    : {'car': 'Toyeca', 'laps': ['00:32.073', '00:32.073', '00:32.073', '00:32.073', '00:32.073', '00:32.073', '00:32.073']},
        'Laggeerok': {'car': 'Toyeca', 'laps': ['00:32.094', '00:32.094', '00:32.094', '00:32.094', '00:32.094', '00:32.094', '00:32.094']},
        'V'        : {'car': 'Toyeca', 'laps': ['00:32.580', '00:32.580', '00:32.580', '00:32.580', '00:32.580', '00:32.580', '00:32.580']},
        'Erzu'     : {'car': 'Cougar', 'laps': ['00:32.777', '00:32.777', '00:32.777', '00:32.777', '00:32.777', '00:32.777', '00:32.777']}
    }
}

In [2]:
#datetime.date(2025, 6, 30) > datetime.datetime.today().date()


In [3]:
import datetime

class CreateChampionshipAlreadyExistingException(Exception):
    MESSAGE = 'The selected championship "{championship_label}" already exists. Change championship name and resubmit it.'
    def __init__(self, label: str):
        super().__init__(self.MESSAGE.format(championship_label=label))

class CreateNonRegisteredTracksException(Exception):
    MESSAGE = 'The selected tracks [{tracks_list_str}] are not registered as tracks for championships. Change tracks among the available ones or register these as new championship tracks.'
    def __init__(self, tracks: set[str]):
        super().__init__(self.MESSAGE.format(tracks_list_str=', '.join(tracks)))

class ChampionshipNotFoundException(Exception):
    MESSAGE = 'The selected championship "{championship_label}" cannot be found in the list of subscribed championships. Change championship label or create a new championship with that label.'
    def __init__(self, label: str):
        super().__init__(self.MESSAGE.format(championship_label=label))

class CreateDuplicateTracksException(Exception):
    MESSAGE = 'The selected tracks for the championship include the following duplicates: [{tracks_list_str}]. Either remove track duplicates, or set "allow_duplicate_tracks" optional parameter to "True".'
    def __init__(self, tracks: list[str]):
        super().__init__(self.MESSAGE.format(tracks_list_str=', '.join(tracks)))

class DeleteTracksNotFoundException(Exception): 
    MESSAGE = 'Could not delete the following tracks: [{tracks_list_str}]. Either ensure these tracks are available for the selected championship, or set "allow_non_existing_tracks" optional parameter to "True".'
    def __init__(self, tracks: list[str]):
        super().__init__(self.MESSAGE.format(tracks_list_str=', '.join(tracks)))

class DeleteTrackPosNotFoundException(Exception):
    MESSAGE = 'Could not delete track in position {track_pos}. The maximum track position value to be deleted for this championship is {max_track_pos}. Either fix the indices for the track positions to be removed, or set the "allow_non_existing_tracks_pos" optional parameter to "True".'
    def __init__(self, track_pos: int, max_track_pos: int):
        super().__init__(self.MESSAGE.format(track_pos=track_pos, max_track_pos=max_track_pos))

class DeleteTracksTypeNotRecognizedException(Exception):
    MESSAGE = 'The required list of tracks to be deleted must either contain the labels of the tracks to be removed (list of strings) or their corresponding positions (list of integers) in the list of tracks for the specified championship. Instead the passed list contains object of {wrong_type} type. Fix the list to be either a list of strings or a list of integers.'
    def __init__(self, wrong_type: object):
        super().__init__(self.MESSAGE.format(wrong_type=wrong_type.__str__))

class CreateTrackPosBiggerThanLengthException(Exception):
    MESSAGE = 'The entered track position "{pos}" for the track to be added to the championship is bigger than the length of tracks list "{max_tracks_len}" for that championship +1. Lower the position or, if you are attempting to add it to the end of the list, avoid passing the track position for insertion.'
    def __init__(self, pos: int, max_tracks_len: int):
        super().__init__(self.MESSAGE.format(pos=pos, max_tracks_len=max_tracks_len))

class CreateChampionshipDateException(Exception):
    MESSAGE = 'The entered startdate "{startdate}" must happen earlier than the entered enddate "{enddate}" for the championship. '
    def __init__(self, startdate: datetime.date, enddate: datetime.date):
        super().__init__(self.MESSAGE.format(startdate=startdate, enddate=enddate))

class AddRacePosOutOfBoundsException(Exception):
    MESSAGE = 'The selected position "{pos}" for the race to be added is out of bounds between 1 (first element of races list) and {races_lst_len} (length of the races list).'
    def __init__(self, pos: int, races_lst_len: int):
        super().__init__(self.MESSAGE.format(pos=pos, races_lst_len=races_lst_len))

class DatabaseConnectionException(Exception):
    MESSAGE = 'Could not establish a connection to remote database due to the following issue: \n {error_message}'
    def __init__(self, error_message: str):
        super().__init__(self.MESSAGE.format(error_message=error_message))

In [4]:
import datetime
from pydantic import BaseModel

#import sys
#import logging
#logger = logging.getLogger()
#logger.setLevel(logging.INFO)
#logger.addHandler(logging.StreamHandler(stream=sys.stdout))

class Player(BaseModel):
    id: str
    label: str

class Car(BaseModel):
    id: str
    label: str

class Racer(BaseModel):
    player: Player
    car: Car

class Track(BaseModel):
    id: str
    label: str
    laptime: datetime.timedelta

    def __str__(self): 
        return self.label

class Race(BaseModel):
    track: Track
    racers: list[Racer]
    duration: datetime.timedelta = datetime.timedelta(minutes=45)
    
    def __str__(self):
        return '{track} (laps: {numlaps})'.format(
            track=self.track.__str__().ljust(20, ' '), 
            numlaps=str(round(self.duration / self.track.laptime)).rjust(3, ' ')
        )

class Championship(BaseModel):
    id: str
    label: str
    races: list[Race]
    startdate: datetime.date
    enddate: datetime.date

    def create_races(self, track_labels: list[str], track_durations: list[datetime.timedelta], allow_duplicate_tracks: bool = False) -> list[Race]:
        if not allow_duplicate_tracks:
            tracks_with_duplicates = [el for el in track_labels if track_labels.count(el) > 1]
            if len(tracks_with_duplicates) > 0:
                raise CreateDuplicateTracksException(tracks_with_duplicates)
        
        if len(track_durations) == 1:
            track_durations *= len(track_labels)
    
        for label, duration in zip(track_labels, track_durations):
            self.add_race(label, duration, allow_duplicate_tracks=True) # allow_duplicate_tracks was already checked for all tracks
            print(f'{self.id} > created empty race on track "{label}"')

        return self.races

    def remove_races_by_track_id(self, track_ids: list[str] = None, allow_non_existing_tracks: bool = False) -> list[Race]:
        if not allow_non_existing_tracks:
            missing_track_ids = set(track_ids).difference(set(map(lambda race: race.track.id, self.races)))
            if len(missing_track_ids) > 0:
                raise DeleteTracksNotFoundException(missing_track_ids)

        for track_id in track_ids:
            self.remove_race(track_id)
            print(f'{self.id} > removed race on track "{track_id}".')

        return self.races

    def remove_races_by_track_pos(self, track_pos: list[int] = None, allow_non_existing_tracks_pos: bool = False) -> list[Race]:
        if not allow_non_existing_tracks_pos:
            if max(track_pos) > len(self.races):
                raise DeleteTrackPosNotFoundException(max(track_pos), len(self.races))

        track_ids = [self.races[pos-1].track.id for pos in track_pos]
        for track_id, track_pos in zip(track_ids, track_pos):
            self.remove_race(track_id, track_pos)
            print(f'{self.id} > removed race on track "{track_id}" in position {track_pos}.')
        
        return self.races
    
    def add_race(self, track_id: str, track_duration: datetime.timedelta, track_pos: int = -1, allow_duplicate_tracks: bool = False) -> list[Race]:
        if not allow_duplicate_tracks:
            if track_id in [race.track.id for race in self.races]:
                raise CreateDuplicateTracksException([track_id])

        if track_pos > len(self.races)+1:
            raise CreateTrackPosBiggerThanLengthException(track_pos, len(self.races))

        new_race = Race(track=available_tracks[track_id], duration=track_duration, racers=[])
        if track_pos == -1:
            self.races.append(new_race)
        else:
            self.races.insert(track_pos-1, new_race)

        return self.races

    def remove_race(self, track_id: str, track_pos: int = -1):
        if track_pos == -1:
            self.races = list(filter(lambda race: race.track.id!=track_id, self.races))
        else:
            self.races.pop(track_pos-1)

    def __str__(self):
        return '\n'.join([
            '',
            '#'*60,
            f' {self.label}',
            '',
            f' Start Date: {self.startdate.strftime("%d %B %Y")}',
            f' End Date: {self.enddate.strftime("%d %B %Y")}',
            '',
            ' Tracks:',
            '\n'.join([
                '   {i}) {race}'.format(i=str(idx+1).rjust(2, ' '), race=race)
                for idx, race in enumerate(self.races)
            ]),
            '',
            '#'*60,
            ''
        ])

def create_championship(id: str, label: str, startdate: datetime.date, enddate: datetime.date) -> str:
    if id in championships.values():
        raise CreateChampionshipAlreadyExistingException(id)

    if startdate > enddate:
        raise CreateChampionshipDateException(startdate, enddate)
        
    championships[id] = Championship(id=id, label=label, races=[], startdate=startdate, enddate=enddate)
    print(f'{id} > created empty championship')
    return label

def create_championship_races(championship_id: str, tracks: list[str], track_durations: list[datetime.timedelta]=[datetime.timedelta(minutes=45)]) -> list[Race]:
    if championship_id not in championships.keys():
        raise ChampionshipNotFoundException(championship_id)

    missing_tracks = set(tracks).difference(available_tracks.keys())
    if len(missing_tracks) > 0:
        raise CreateNonRegisteredTracksException(missing_tracks)

    return championships[championship_id].create_races(tracks, track_durations)

def add_championship_race(championship_id: str, track: str, duration: datetime.timedelta=datetime.timedelta(minutes=45), pos: int = -1) -> list[Race]:
    if championship_id not in championships.keys():
        raise ChampionshipNotFoundException(championship_label)

    if track not in available_tracks.keys():
        raise CreateNonRegisteredTracksException([track])

    if not ((pos == -1) or (pos > 0 and pos < len(championships[championship_id].races)+2)):
        raise AddRacePosOutOfBoundsException(pos, len(championships[championship_id].races)+1)
    
    pos = pos if pos > -1 else len(championships[championship_id].races)+1
    updated_races = championships[championship_id].add_race(track, duration, pos)
    print(f'{championship_id} > created empty race on track "{track}" in position {pos}')

    return updated_races

def remove_championship_races(championship_id: str, tracks: list[str|int]) -> str:
    if championship_id not in championships.keys():
        raise ChampionshipNotFoundException(championship_id)
    championship = championships[championship_id]
    
    if len(tracks) == 0:  
        print(f'{championship.id} > deleted no track because of empty list passed as parameter.')
        return championship.races
        
    if isinstance(tracks[0], str):
        remaining_races = championship.remove_races_by_track_id(tracks)
    elif isinstance(tracks[0], int):
        remaining_races = championship.remove_races_by_track_pos(sorted(tracks, reverse=True))
    else:
        raise RemoveTracksTypeNotRecognizedException(type(tracks[0]))
        
    return remaining_races

#def subscribe_player_to_championship(player_id: str, championship_id: str):

available_tracks = {
    'museum2'        : Track(id='museum2'        , label='Museum 2'         , laptime=datetime.timedelta(hours=0, minutes=0, seconds=34, milliseconds=872)),
    'portlimano2'    : Track(id='portlimano2'    , label='Port Limano 2'    , laptime=datetime.timedelta(hours=0, minutes=0, seconds=39, milliseconds=733)),
    'gameroom2'      : Track(id='gameroom2'      , label='Game Room 2'      , laptime=datetime.timedelta(hours=0, minutes=0, seconds=37, milliseconds=507)),
    'spavolt2'       : Track(id='spavolt2'       , label='Spa Volt 2'       , laptime=datetime.timedelta(hours=0, minutes=0, seconds=39, milliseconds=194)),
    'venice'         : Track(id='venice'         , label='Venice'           , laptime=datetime.timedelta(hours=0, minutes=0, seconds=41, milliseconds= 33)),
    'downtown1'      : Track(id='downtown1'      , label='Downtown 1'       , laptime=datetime.timedelta(hours=0, minutes=0, seconds=46, milliseconds=569)),
    'spavolt1'       : Track(id='spavolt1'       , label='Spa Volt 1'       , laptime=datetime.timedelta(hours=0, minutes=0, seconds=45, milliseconds=233)),
    'hospital1'      : Track(id='hospital1'      , label='Hospital 1'       , laptime=datetime.timedelta(hours=0, minutes=0, seconds=37, milliseconds=599)),
    'santorini'      : Track(id='santorini'      , label='Santorini'        , laptime=datetime.timedelta(hours=0, minutes=0, seconds=31, milliseconds=585)),
    'whiterosechapel': Track(id='whiterosechapel', label='White Rose Chapel', laptime=datetime.timedelta(hours=0, minutes=0, seconds=48, milliseconds=267)),
}

available_players = {
    'ahma'   : Player(id='ahma'  , label='Ahma'        ),
    'ashen'  : Player(id='ashen' , label='Ashen Forest'),
    'bgm'    : Player(id='bgm'   , label='BGM'         ),
    'c'      : Player(id='c'     , label='C'           ),
    'duc'    : Player(id='duc'   , label='ducsekbence' ),
    'erzu'   : Player(id='erzu'  , label='Erzu'        ),
    'dolo'   : Player(id='dolo'  , label='Dolo'        ),
    'famous' : Player(id='famous', label='Famous'      ),
    'floxit' : Player(id='floxit', label='Floxit'      ),
    'frost'  : Player(id='frost' , label='Frosttbitten'),
    'g'      : Player(id='g'     , label='G'           ),
    'ganesh' : Player(id='ganesh', label='Ganesh'      ),
    'gforce' : Player(id='gforce', label='GForce'      ),
    'iurac'  : Player(id='iurac' , label='Iurac'       ),
    'kiki'   : Player(id='kiki'  , label='Kiki'        ),
    'kila'   : Player(id='kila'  , label='Kilabarus'   ),
    'kipy'   : Player(id='kipy'  , label='Kipy'        ),
    'klnfln' : Player(id='klnfln', label='KlnFln'      ),
    'lagge'  : Player(id='laggee', label='Laggeerok'   ),
    'lamp'   : Player(id='lamp'  , label='Lampaert'    ),
    'lopes'  : Player(id='lopes' , label='LoPesca'     ),
    'macaco' : Player(id='macaco', label='Macacosky'   ),
    'maty'   : Player(id='maty'  , label='Maty07'      ),
    'nick'   : Player(id='nick'  , label='Nickurn'     ),
    'ohnej'  : Player(id='ohnej' , label='OhNej'       ),
    'power'  : Player(id='power' , label='Powerate'    ),
    'sebr'   : Player(id='sebr'  , label='SebR'        ),
    'shige'  : Player(id='shige' , label='Shigekix'    ),
    'sting'  : Player(id='sting' , label='Stingox'     ),
    'supers' : Player(id='supers', label='Supersajdzan'),
    'tova'   : Player(id='tova'  , label='Tova'        ),
    'tt'     : Player(id='tt'    , label='TT'          ),
    'tubers' : Player(id='tubers', label='Tubers'      ),
    'v'      : Player(id='v'     , label='V'           ),
    'vasosk' : Player(id='vasosk', label='Vasosky'     ),
    'vlad'   : Player(id='vlad'  , label='Vlad'        ),
    'wurzel' : Player(id='wurzel', label='Wurzel'      ),
    'zeino'  : Player(id='zeino' , label='Zeino'       ),
}

available_cars = {
    'amw'   : Car(id='amw'   , label='AMW'   ),
    'cougar': Car(id='cougar', label='Cougar'),
    'humma' : Car(id='humma' , label='Humma' ),
    'toyeca': Car(id='toyeca', label='Toyeca'),
}

championships = {}
tracks = [ 'museum2', 'portlimano2', 'gameroom2', 'spavolt2', 'venice', 'downtown1', 'spavolt1', 'hospital1', 'santorini', 'whiterosechapel']
create_championship(id='rvec2025', label='Re-Volt Endurance Championship 2025', startdate=datetime.date(2025, 6, 30), enddate=datetime.date(2025, 7, 30))
create_championship_races(championship_id='rvec2025', tracks=tracks)
remove_championship_races(championship_id='rvec2025', tracks=['venice', 'spavolt1'])
add_championship_race(championship_id='rvec2025', track='venice', pos=9)
add_championship_race(championship_id='rvec2025', track='spavolt1', pos=2)

    
print(championships['rvec2025'])

rvec2025 > created empty championship
rvec2025 > created empty race on track "museum2"
rvec2025 > created empty race on track "portlimano2"
rvec2025 > created empty race on track "gameroom2"
rvec2025 > created empty race on track "spavolt2"
rvec2025 > created empty race on track "venice"
rvec2025 > created empty race on track "downtown1"
rvec2025 > created empty race on track "spavolt1"
rvec2025 > created empty race on track "hospital1"
rvec2025 > created empty race on track "santorini"
rvec2025 > created empty race on track "whiterosechapel"
rvec2025 > removed race on track "venice".
rvec2025 > removed race on track "spavolt1".
rvec2025 > created empty race on track "venice" in position 9
rvec2025 > created empty race on track "spavolt1" in position 2

############################################################
 Re-Volt Endurance Championship 2025

 Start Date: 30 June 2025
 End Date: 30 July 2025

 Tracks:
    1) Museum 2             (laps:  77)
    2) Spa Volt 1           (laps:  6

In [8]:
import pandas as pd
from sqlalchemy import create_engine, text

class Database:
    CONN_STRING = 'postgresql+psycopg2://{username}:{password}@{hostname}/{database}'
    
    cars: list[Car]
    players: list[Player]
    tracks: list[Track]

    def __init__(self, db_name: str, debug: int = 0):
        self.__init_conn__()
        self.db_name = 'rv_world'
        self.debug = debug

    def __init_conn__(self, user: str='rv_admin', pwd: str='rv_p4ssw0rd', host: str='localhost', db: str='rv_world'):
        self.engine = create_engine(
            self.CONN_STRING.format(username=user, password=pwd, hostname=host, database=db)
        )
        self.conn = self.engine.connect()
        
    def __test_conn__(self):
        pd.read_sql('SELECT 1', self.conn)

    def __execute_stmt__(self, stmt: str, attempt: int = 0, max_num_attempts: int = 10):
        try:
            self.conn.execute(text(stmt))
            self.conn.commit()
        except Exception as e:
            if attempt < max_num_attempts:
                self.__init_conn__()
                self.__execute_stmt__(stmt, 1)
            else:
                self.conn.close()
                self.engine.dispose()
                raise DatabaseConnectionException(e) 
            
    ######## TABLES
    def create_table(self, table_name: str, cols: dict[str, dict[str, str|list[str]]], pkeys: list[str] = [], fkeys: dict[str, dict[str, str]] = [], force: bool = False, debug: int = 0):
        if debug:
            print('{db_name} db > creating table "{table_name}"{force}'.format(
                db_name=self.db_name,
                table_name=table_name,
                force=' (force)' if force else ''
            ))
        if force:
            self.drop_table(table_name, debug=False)
            
        self.__execute_stmt__(
            'CREATE TABLE IF NOT EXISTS {table_name}('.format(table_name=table_name.lower().capitalize()) 
          + '\n' + ',\n'.join(['    {col_name} {dtype}{modifiers}'.format(
              col_name=col_name.lower(),
              dtype=col_info['dtype'].upper(),
              modifiers=' '+' '.join([mod.upper() for mod in col_info['modifiers']])
          ) for col_name, col_info in cols.items()])
          + (',\n' + '    PRIMARY KEY ({primary_keys})'.format(primary_keys=', '.join([k.lower() for k in pkeys])) if len(pkeys) > 0 else '')
          + (',\n' + ',\n'.join({'    FOREIGN KEY ({col}) REFERENCES {ext_tab}({ext_col}) ON DELETE SET NULL'.format(
              col=col,
              ext_tab=ext_ref['tab'].lower().capitalize(),
              ext_col=ext_ref['col'].lower()
          ) for col, ext_ref in fkeys.items()}) if len(fkeys) > 0 else '')
          + '\n' + ');'
          + '\n'
        )
        return self.read_table(table_name='information_schema.tables', conditions={'table_schema': 'public', 'table_catalog': 'rv_world'})

    def drop_table(self, table_name: str, debug: int = 0):
        if debug:
            print('{db_name} db > deleting table "{table_name}"'.format(
                db_name=self.db_name,
                table_name=table_name
            ))
        self.__execute_stmt__(
            'DROP TABLE IF EXISTS {table_name}'.format(table_name=table_name.lower().capitalize())
          + ';'
          + '\n'
        )
        return self.read_table(table_name='information_schema.tables', conditions={'table_schema': 'public', 'table_catalog': 'rv_world'})

    ######## VALUES
    def insert_into_table(self, table_name: str, cols: list[str], records: list[list[str]], debug: int = 0):
        if debug:
            print('{db_name} db > inserting {n_entries} entries in the table "{table_name}"'.format(
                db_name=self.db_name,
                n_entries=len(records),
                table_name=table_name
            ))
        self.__execute_stmt__(
            'INSERT INTO {table_name}({col_names})'.format(table_name=table_name.lower().capitalize(), col_names=', '.join([col.lower() for col in cols]))
          + '\n' + 'VALUES'
          + '\n' + ',\n'.join(['    ({values})'.format(values=', '.join(
              [f'{v}' if isinstance(v, int) else f'\'{v}\'' for v in lst]
          )) for lst in records])
          + ';'
          + '\n'
        )
        return self.read_table(table_name=table_name)

    def read_table(self, table_name: str, columns: list[str] = [], conditions: dict = {}, debug: int = 0) -> pd.DataFrame:
        if debug:
            print('{db_name} db > reading {n_entries} entries from the table "{table_name}"'.format(
                db_name=self.db_name,
                n_entries=self.read_table(table_name, columns=['SUM(1)'], conditions=conditions)[0].tolist()[0],
                table_name=table_name
            ))
        return pd.read_sql(\
              'SELECT {columns}'.format(columns=', '.join(columns) if len(columns) > 0 else '*') 
            + '\n' + 'FROM {table_name}'.format(table_name=table_name)
            + ('\nWHERE \n' + '{conditions}'.format(conditions=' AND \n'.join(
                ['    {col} = {val}'.format(col=k, val=f'{v}' if isinstance(v, int) else f'\'{v}\'') for k, v in conditions.items()]
            )) if len(conditions) > 0 else '')
            + ';'
            + '\n'
            , self.conn
        )

    def update_table(self, table_name: str, updating_values: dict, conditions: dict = {}, debug: int = 0):
        if debug:
            print('{db_name} db > updating {n_entries} entries in the table "{table_name}"'.format(
                db_name=self.db_name,
                n_entries=self.read_table(table_name, columns=['SUM(1)'], conditions=conditions)[0].tolist()[0],
                table_name=table_name
            ))
        self.__execute_stmt__(
            'UPDATE {table_name}'.format(table_name=table_name.lower().capitalize())
          + '\nSET \n' + '{setters}'.format(setters=',\n'.join(
              ['    {col} = {val}'.format(col=k, val=f'{v}' if isinstance(v, int) else f'\'{v}\'') for k, v in updating_values.items()]
          ))
          + ('\nWHERE \n' + '{conditions}'.format(conditions=' AND \n'.join(
              ['    {col} = {val}'.format(col=k, val=f'{v}' if isinstance(v, int) else f'\'{v}\'') for k, v in conditions.items()]
          )) if len(conditions) > 0 else '')
          + ';'
          + '\n'
        )
        return self.read_table(table_name=table_name, conditions=conditions)

    def delete_from_table(self, table_name: str, conditions: dict = {}, debug: int = 0):
        if debug:
            print('{db_name} db > deleting {n_entries} entries in the table "{table_name}"'.format(
                db_name=self.db_name,
                n_entries=self.read_table(table_name, columns=['SUM(1)'], conditions=conditions)[0].tolist()[0],
                table_name=table_name
            ))
        self.__execute_stmt__(
            'DELETE FROM {table_name}'.format(table_name=table_name.lower().capitalize())
          + ('\nWHERE \n' + '{conditions}'.format(conditions=' AND \n'.join(['    {col} = {val}'.format(col=k, val=f'{v}' if isinstance(v, int) else f'\'{v}\'') for k, v in conditions.items()])) if len(conditions) > 0 else '')
          + ';'
          + '\n'
        )
        return self.read_table(table_name=table_name, conditions=conditions)


import json 

class RVWorldDB(Database):

    def __init__(self):
        super(RVWorldDB, self).__init__(db_name='rv_world')

        with open('{db_name}.json'.format(db_name=self.db_name), 'r') as handle:
            self.db_info = json.load(handle)
        
    
    def create_tables(self, force: bool = True):
        for tab in [k for k in self.db_info['tables'].keys()][::-1]:
            db.drop_table(table_name=tab)
        for table_name, table_info in self.db_info['tables'].items():
            self.create_table(
                table_name=table_name,
                cols=table_info['columns'],
                pkeys=table_info['pkeys'],
                fkeys=table_info['fkeys'],
                force=force,
                debug=True
            )
        self.conn.commit()
    
    def load_cars(self):
        pass

    def load_players(self):
        pass

    def load_tracks(self):
        pass

db = RVWorldDB()
db.create_tables()

rv_world db > creating table "Track" (force)
rv_world db > creating table "Player" (force)
rv_world db > creating table "Car" (force)
rv_world db > creating table "Championship" (force)
rv_world db > creating table "Race" (force)
rv_world db > creating table "Racer" (force)


In [ ]:
db.delete_table(table_name='cars')
db.create_table(table_name='cars', cols={'id': 'serial', 'name': 'varchar(60)'}, pkeys=['id'])
db.insert_into_table(table_name='cars', cols=['name'], values=[['amw'], ['humma'], ['cougar'], ['toyeca']])
db.read_table(table_name='cars')

In [ ]:
def add_race_results():
   pass

In [ ]:
championships

In [ ]:
def create_championship():
   pass

def create_race_results():
   pass

def read_championship():
   pass

def read_race_results():
   pass

def update_race_results():
   pass

def delete_race_results():
   pass

def delete_championship():
   pass


In [ ]:
import pandas as pd

pd.read_csv('session_2024-08-17_15-18-39.csv')

In [ ]:
import json

with open("race_results.json", "w") as file: 
    json.dump(race_results, file)

In [ ]:
import pandas as pd
import re

df = pd.DataFrame(
    [[k]+[int(t) for t in re.split(r'[.:]', v)] for k, v in race_results.items()],
    columns=['player', 'hours', 'minutes', 'seconds', 'milliseconds']
)
df['time'] = df['milliseconds'] + 1000*df['seconds'] + 60000*df['minutes'] + 3600000*df['hours']
df = df.sort_values('time')

df['timegap'] = df['time'] - df['time'].shift()

# adding features

df

In [ ]:


#df['gap']

In [ ]:
import pandas as pd
import pickle

out_race = pd.DataFrame([
    [ 1,   'Stingox', '45:16.280',        None,        None, '00:31.333', '00:31.585', 99.20, 'Toyeca', 25],
    [ 2,    'Famous', '45:24.412', '00:08.132', '00:08.132', '00:31.394', '00:31.679', 99.10, 'Toyeca', 20],
    [ 3,      'Kiki', '45:27.048', '00:10.768', '00:02.636', '00:31.446', '00:31.710', 99.17, 'Toyeca', 16],
    [ 4, 'Kilabarus', '45:27.807', '00:11.527', '00:00.759', '00:31.329', '00:31.719', 98.77, 'Toyeca', 13],
    [ 5,  'Powerate', '45:34.066', '00:17.786', '00:06.259', '00:31.499', '00:31.791', 99.08, 'Toyeca', 11],
    [ 6,    'GForce', '45:37.610', '00:21.330', '00:03.544', '00:31.364', '00:31.833', 98.53, 'Toyeca', 10],
    [ 7, 'Macacosky', '45:40.227', '00:23.947', '00:02.617', '00:31.430', '00:31.863', 98.64, 'Toyeca',  9],
    [ 8,      'Ahma', '45:42.519', '00:26.239', '00:02.292', '00:31.452', '00:31.890', 98.63, 'Toyeca',  8],
    [ 9,   'Nickurn', '45:46.241', '00:29.961', '00:03.722', '00:31.189', '00:31.933', 97.67, 'Toyeca',  7],
    [10,  'Lampaert', '45:54.066', '00:37.786', '00:07.825', '00:31.545', '00:31.024', 98.50, 'Toyeca',  6],
    [11,     'Iurac', '45:58.320', '00:42.040', '00:04.254', '00:31.361', '00:31.073', 97.78, 'Toyeca',  5],
    [12, 'Laggeerok', '46:00.046', '00:43.766', '00:01.726', '00:31.749', '00:31.094', 98.93, 'Toyeca',  4],
    [13,         'V', '46:41.874', '01:25.594', '00:41.828', '00:31.511', '00:31.580', 96.72, 'Toyeca',  3],
    [14,      'Erzu', '46:58.827', '01:42.547', '00:16.953', '00:31.896', '00:31.777', 97.31, 'Cougar',  2],
], columns=[
    'pos', 'player',
    'race_time', 'gap',
    'interval', 'best_lap', 'average_lap',
    'consistency',
    'car',
    'points'
])

with open('out_race.pickle', 'wb') as handle:
    pickle.dump(out_race, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
out_rvec = pd.DataFrame([
    [ 1,      'Stingox', 21, 16, 20, 20,  5, 13, 26, 21, 25, None, 149, 167],
    [ 2,       'Famous', 25, 10, 25, 16, 25, 16, 11, 16, 20, None, 143, 164],
    [ 3,    'Kilabarus',  8, 25, 16, 25, 11, 25, 13, 13, 13, None, 130, 149],
    [ 4,         'Kiki', 16, 20, 14,  0, 13, 20,  4, 25, 16, None, 124, 128],
    [ 5,      'Nickurn', 13, 14, 10,  8, 20, 11,  6,  3,  8, None,  84,  93],
    [ 6,       'GForce', 10,  9,  9, 13, 10,  9, 20, 11, 10, None,  83, 101],
    [ 7,    'Macacosky', 11,  2,  6, 10,  0,  6, 16,  9,  9, None,  67,  69],
    [ 8, 'Frosttbitten',  7, 11,  0,  0, 16, 11, 10, 10,  0, None,  65,  65],
    [ 9,     'Powerate',  9,  8,  8,  9,  2,  8,  9,  8, 11, None,  62,  72],
    [10,     'Lampaert',  5,  6,  7, 11,  6,  0,  5,  0,  6, None,  46,  46],
    [11,         'Ahma',  0,  5, 11,  0,  9,  4,  2,  6,  8, None,  45,  45],
    [12,    'Laggeerok',  3,  0,  3,  4,  0,  7,  8,  7,  4, None,  36,  36],
    [13,         'Erzu',  0,  7,  0,  8,  5,  0,  7,  5,  2, None,  34,  34],
    [14, 'Ashen Forest',  2,  4,  5,  5,  7,  2,  0,  0,  0, None,  25,  25],
    [15,            'V',  4,  0,  0,  0,  0,  5,  3,  0,  3, None,  15,  15],
    [16,        'Ohnej',  6,  3,  4,  0,  0,  0,  0,  0,  0, None,  13,  13],
    [17,        'Iurac',  0,  0,  0,  6,  0,  0,  0,  0,  5, None,  11,  11],
    [18,         'Kipy',  0,  0,  0,  0,  8,  0,  0,  0,  0, None,   8,   8],
    [19,       'Wurzel',  0,  0,  0,  0,  3,  3,  0,  0,  0, None,   6,   6],
    [20,     'Shigekix',  0,  0,  0,  0,  0,  0,  0,  4,  0, None,   4,   4],
    [21,       'Klnfln',  1,  0,  0,  0,  0,  0,  0,  0,  0, None,   1,   1],
    [22,       'Maty07',  0,  1,  0,  0,  0,  0,  0,  0,  0, None,   1,   1],
], columns=[
    'pos', 'player',
    'museum1', 'portlimano2', 'gameroom2', 'spavolt2', 'venice', 
    'downtown1', 'spavolt1', 'hospital1', 'santorini', 'whiterosechapel',
    'total', 'all_tracks'
])


with open('out_rvec.pickle', 'wb') as handle:
    pickle.dump(out_rvec, handle, protocol=pickle.HIGHEST_PROTOCOL)